### IMPORTS

In [2]:
import pymc as pm
import numpy as np
import pandas as pd
import arviz as az
import matplotlib.pyplot as plt
import scipy.stats as stats

### QUESTION 1

In [ ]:
def metropolis_hastings(function, draws):
    trace = np.zeros(draws)
    
    xCurrent = 0.5
    probCurrent = func.pdf(xCurrent)
    delta = np.random.normal(0, 0.5, draws)
    
    for i in range(draws):
        xNew = xCurrent + delta[i]
        probNew = func.pdf(xNew)
        acceptance_rate = probNew / probCurrent
        
        if acceptance_rate >= np.random.random():
            trace[i] = xNew
            xCurrent = xNew
            probCurrent = probNew

        else:
            trace[i] = xCurrent

    return trace

func = stats.beta(2, 5) 
trace = metropolis(func=func) 

### QUESTION 2

In [8]:
problem2 = pd.read_csv('problem2.csv')

In [9]:
problem2.head()

,x1,x2,y
0,0.77,0.19,10.84
1,0.10,0.45,8.75
2,0.58,0.64,9.93
3,0.76,0.76,10.90
4,0.84,0.49,10.45


In [ ]:
with pm.Model() as p2:

    # DATA
    x_i1 = pm.Data(name="x_i1",
                        value=problem2['x1'],
                        mutable=False)
    x_i2 = pm.Data(name="x_i2",
                        value=problem2['x2'],
                        mutable=False)

    y = pm.Data(name="y",
                        value=problem2['y'],
                        mutable=False)

    # NONINFORMATIVE PRIORS
    beta0 = pm.Normal(name="beta0",
                      mu=0,
                      tau=0.001)
    beta1 = pm.Normal(name="beta1",
                      mu=0,
                      tau=0.001)
    beta2 = pm.Normal(name="beta2",
                      mu=0,
                      tau=0.001)
    alpha0 = pm.Normal(name="alpha0",
                      mu=0,
                      tau=0.001)
    alpha1 = pm.Normal(name="alpha1",
                      mu=0,
                      tau=0.001)
    alpha2 = pm.Normal(name="alpha2",
                      mu=0,
                      tau=0.001)

    # GETTING EPSILON
    sigma_sq = pm.math.exp(alph0 + (alpha1 * x_i1) + (alpha2 * x_i2))
    tau = 1 / sigma_sq
    epsilon_i = pm.Normal(name='epsilon_i', mu=0, tau=tau)

    # GETTING PREDICTED Y
    mu = beta0 + (beta1 * x_i1) + (beta2 * x_i2) + epsilon_i
    y_i = pm.Normal(name='y_i', mu=var, tau=0.001, observed=y)

    trace_p2 = pm.sample(draws=10000, tune=1000, cores=None, chains=4)

In [ ]:
az.summary(trace_p2, hdi_prob=0.95)

In [ ]:
ppc = pm.sample_posterior_predictive(trace, model=p2, predictions=True)

In [ ]:
y_pred2 = trace_p2.posterior_predictive.stack(sample=("chain", "draw"))["y_i"].values.T
az.r2_score(y, y_pred2)

### QUESTION 3

In [10]:
problem3 = pd.read_csv('problem3.csv')

In [11]:
problem3.head()

,x,y
0,0.988,0
1,0.222,1
2,0.285,3
3,0.326,3
4,0.140,2


In [ ]:
with pm.Model() as p3:

    x = pm.Data(name="x",
                       value=problem3["x"],
                       mutable=False)

    y = pm.Data(name="y",
                   value=problem3["y"],
                   mutable=False)

    beta0 = pm.Normal("beta0", mu=0, tau=0.0001)
    beta1 = pm.Normal('beta1', mu=0, tau=0.0001)
    beta2 = pm.Normal('beta2', mu=0, tau=0.0001)

    regression = beta0 + pm.math.dot(l=x, r=beta1) + pm.math.dot(l=x**2, r=beta2)
    lambda_xi = pm.math.exp(regression)

    y_i = pm.Poisson('y_i', mu=lambda_xi, observed=y)

    trace_p3 = pm.sample(draws=10000, tune=1000, cores=None, chains=4)

In [ ]:
az.summary(trace_p3, hdi_prob=0.95)

In [ ]:
ppc = pm.sample_posterior_predictive(trace_p3, model=p3, predictions=True)

In [ ]:
y_pred3 = trace_p3.posterior_predictive.stack(sample=("chain", "draw"))["y_i"].values.T
az.r2_score(y, y_pred3)